# Impacto das Tarifas Americanas nas Economias Parceiras

## 🚀 Projeto: "Impacto das Tarifas Americanas nas Economias Parceiras"

### 🎯 Objetivo:
O objetivo deste projeto é analisar o impacto das tarifas impostas pelos EUA em seus parceiros comerciais, e como esses impactos afetam a balança comercial (déficit/superávit) e a população dos países. Utilizando PySpark, vamos calcular a relação entre as tarifas, a população e a economia de cada país, e classificar os países em categorias de risco econômico.

### ✅ O que será implementado:

1. **Calcular o "Impacto Tarifa/População"**
Vamos adicionar uma coluna ao nosso conjunto de dados que mostre o impacto das tarifas em relação à população de cada país. Esse cálculo nos ajuda a entender se países menores estão sendo mais impactados proporcionalmente pelas tarifas dos EUA.

```python
from pyspark.sql.functions import col
```

# Calculando o impacto da tarifa sobre a população
```python
df = df.withColumn("Tariff Impact", col("Trump Tariffs Alleged") / col("Population"))
```
Classificar países em categorias de risco econômico Com base na análise do déficit/superávit da balança comercial dos EUA e o nível de tarifas impostas aos países, vamos classificar os países em diferentes categorias de risco econômico. Os países serão classificados como "High Risk" (alto risco), "Medium Risk" (médio risco) ou "Low Risk" (baixo risco).

# Classificando os países de acordo com o risco econômico

```python
df = df.withColumn("Risk Category", when(
    (col("US 2024 Deficit") < 0) & (col("Trump Tariffs Alleged") > 30), "High Risk"
).when(
    (col("US 2024 Deficit") < 0), "Medium Risk"
).otherwise("Low Risk"))
```
🔧 Como executar:
Pré-requisitos:

Ter o PySpark instalado e configurado.

Dados de tarifas dos EUA, déficit comercial e população dos países disponivel em Data/Csv/TariffCalculationsplusPopulation.csv.
ou no link https://www.kaggle.com/datasets/danielcalvoglez/us-tariffs-2025

Passos para execução:

Carregue esse Notebook em sua plataforma
. Pycharm, Databricks, Jupyter....

Execute a análise e visualize os resultados.

📊 Resultados Esperados:
Após rodar o código, você terá um DataFrame com informações sobre:

O impacto das tarifas sobre a população de cada país.

A classificação de risco econômico de cada país, com base no déficit comercial e nas tarifas impostas.

🔄 Possíveis Expansões:
Adicionar mais dados econômicos, como PIB, exportações e importações.

Implementar visualizações para melhor entender os impactos econômicos.

Analisar como as respostas dos países (ex: retaliações tarifárias) afetam as categorias de risco.


In [117]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import when

In [42]:
spark = SparkSession.builder.appName("Spark").getOrCreate()

In [43]:
data = [("Ana",23,"SP"),("Carlos",45,"RJ"),("Bruno",36,"BH")]

In [44]:
df = spark.createDataFrame(data)
df.show()

+------+---+---+
|    _1| _2| _3|
+------+---+---+
|   Ana| 23| SP|
|Carlos| 45| RJ|
| Bruno| 36| BH|
+------+---+---+



In [45]:

df = df.withColumnRenamed("_1","Name").withColumnRenamed("_2","Age").withColumnRenamed("_3","City")
df.show()

+------+---+----+
|  Name|Age|City|
+------+---+----+
|   Ana| 23|  SP|
|Carlos| 45|  RJ|
| Bruno| 36|  BH|
+------+---+----+



In [46]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- City: string (nullable = true)



In [48]:
month = 12
df = df.withColumn("age_in_month", col("Age") * month)

In [51]:
df = df.select("Name", "Age","age_in_month", "City")
display(df.show())

+------+---+------------+----+
|  Name|Age|age_in_month|City|
+------+---+------------+----+
|   Ana| 23|         276|  SP|
|Carlos| 45|         540|  RJ|
| Bruno| 36|         432|  BH|
+------+---+------------+----+



None

In [52]:
age_filter_than_30 = df.filter(col("Age") > 30)
age_filter_than_30.show()

+------+---+------------+----+
|  Name|Age|age_in_month|City|
+------+---+------------+----+
|Carlos| 45|         540|  RJ|
| Bruno| 36|         432|  BH|
+------+---+------------+----+



In [112]:
read_options = {
    "header": True,
    "sep": ";",
    "inferSchema": True,
}

df1 = spark.read.csv("./Data/csv/TariffCalculationsplusPopulation.csv",**read_options)
df1.show(6)


+-----------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|    Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|
+-----------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|Afghanistan|          -11.1|           11.4|                           22.6|                  49%|           25%|  41454761|
|    Albania|           13.4|          141.7|                          128.3|                  10%|           10%|   2745972|
|    Algeria|      -1,447.10|       1,014.50|                       2,461.60|                  59%|           29%|  46164219|
|    Andorra|            1.5|            4.9|                            3.4|                  10%|           10%|     80856|
|     Angola|      -1,186.90|          682.4|                       1,869.20|                  63%|           32%|  36

In [113]:
df1.printSchema()

root
 |-- Country: string (nullable = true)
 |-- US 2024 Deficit: string (nullable = true)
 |-- US 2024 Exports: string (nullable = true)
 |-- US 2024 Imports (Customs Basis): string (nullable = true)
 |-- Trump Tariffs Alleged: string (nullable = true)
 |-- Trump Response: string (nullable = true)
 |-- Population: integer (nullable = true)



In [114]:
df1_clean = (
    df1
    .filter(col("Population").isNotNull()).filter(col("Trump Tariffs Alleged").isNotNull())
)

df1_clean = df1_clean.withColumn("US 2024 Deficit", regexp_replace(col("US 2024 Deficit"),",","").cast("float")) \
             .withColumn("US 2024 Exports",regexp_replace(col("US 2024 Exports"),",","").cast("float")) \
             .withColumn("US 2024 Imports (Customs Basis)",regexp_replace(col("US 2024 Imports (Customs Basis)"), ",", "").cast("float"))
display(df1_clean.show())

+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|            Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|
+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|        Afghanistan|          -11.1|           11.4|                           22.6|                  49%|           25%|  41454761|
|            Albania|           13.4|          141.7|                          128.3|                  10%|           10%|   2745972|
|            Algeria|        -1447.1|         1014.5|                         2461.6|                  59%|           29%|  46164219|
|            Andorra|            1.5|            4.9|                            3.4|                  10%|           10%|     80856|
|             Angola|        -1186.9|          682.4|         

None

In [115]:
df1_imports_in_decimal = df1_clean.withColumn("Trump Tariffs Alleged", regexp_replace(col("Trump Tariffs Alleged"),"%", "").cast("float")/ 100)
df1_imports_in_decimal.show(5)

+-----------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|    Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|
+-----------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|Afghanistan|          -11.1|           11.4|                           22.6|                 0.49|           25%|  41454761|
|    Albania|           13.4|          141.7|                          128.3|                  0.1|           10%|   2745972|
|    Algeria|        -1447.1|         1014.5|                         2461.6|                 0.59|           29%|  46164219|
|    Andorra|            1.5|            4.9|                            3.4|                  0.1|           10%|     80856|
|     Angola|        -1186.9|          682.4|                         1869.2|                 0.63|           32%|  36

In [116]:
df_Tarrif_impact_column = df1_imports_in_decimal.withColumn("Tarrif Impact",col("Trump Tariffs Alleged") / col("Population"))
display(df_Tarrif_impact_column.show(6))

+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+--------------------+
|            Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|       Tarrif Impact|
+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+--------------------+
|        Afghanistan|          -11.1|           11.4|                           22.6|                 0.49|           25%|  41454761|1.182011397918805...|
|            Albania|           13.4|          141.7|                          128.3|                  0.1|           10%|   2745972|3.641697730348307E-8|
|            Algeria|        -1447.1|         1014.5|                         2461.6|                 0.59|           29%|  46164219|1.278046098862844...|
|            Andorra|            1.5|            4.9|                 

None

In [121]:
df1_risk_category = df_Tarrif_impact_column.withColumn("Risk Category",when((col("US 2024 Deficit") < 0 ) & (col("Trump Tariffs Alleged") > 30), "High Risk").when((col("US 2024 Deficit") < 0), "Medium Risk").otherwise("Low Risk"))

display(df1_risk_category.show(7))

+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+--------------------+-------------+
|            Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|       Tarrif Impact|Risk Category|
+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+--------------------+-------------+
|        Afghanistan|          -11.1|           11.4|                           22.6|                 0.49|           25%|  41454761|1.182011397918805...|  Medium Risk|
|            Albania|           13.4|          141.7|                          128.3|                  0.1|           10%|   2745972|3.641697730348307E-8|     Low Risk|
|            Algeria|        -1447.1|         1014.5|                         2461.6|                 0.59|           29%|  46164219|1.278046098862844...| 

None